In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Cambios/Modificaciones archivo orginal

Modificaciones realizadas en el archivo de train durante el CHP1, para su debida justificación referirse a *'7506R_TP1_GRUPO32_CHP1_ENTREGA_N1'*

In [2]:
df_trabajo = pd.read_csv('/content/drive/MyDrive/datasetsTP/hotels_train.csv')

## Eliminación valores negativos

In [3]:
columnas_numericas = df_trabajo.select_dtypes(include=['int64', 'float64']).columns

for columna in columnas_numericas:
    negativos_df = df_trabajo[df_trabajo[columna] < 0]
    if not negativos_df.empty:
        print(f"Columna '{columna}' tiene valores negativos:")
        for index, row in negativos_df.iterrows():
            print(f"Indice: {index}, valor: {row[columna]}")
        print("\n")

Columna 'adr' tiene valores negativos:
Indice: 21124, valor: -6.38




In [4]:
adr_negativo_df = df_trabajo[df_trabajo['adr'] < 0]

df_trabajo = df_trabajo.drop(adr_negativo_df.index)

df_trabajo = df_trabajo.reset_index(drop=True)
#Si se corre el código de arriba se puede comprobar la correcta eliminación

## Tratamiento de datos nulos e invalidos

In [5]:
undefined_meals = df_trabajo[df_trabajo['meal'] == 'Undefined']
df_trabajo = df_trabajo.drop(undefined_meals.index)
df_trabajo = df_trabajo.reset_index(drop=True)

undefined_market_segment = df_trabajo[df_trabajo['market_segment'] == 'Undefined']
df_trabajo = df_trabajo.drop(undefined_market_segment.index)
df_trabajo = df_trabajo.reset_index(drop=True)

undefined_distribution_channel = df_trabajo[df_trabajo['distribution_channel'] == 'Undefined']
df_trabajo = df_trabajo.drop(undefined_distribution_channel.index)
df_trabajo = df_trabajo.reset_index(drop=True)

In [6]:
columnas_object = df_trabajo.select_dtypes(include=['object', 'category']).columns

for column in columnas_object:
    value_count = (df_trabajo[column] == 'Undefined').sum()
    if value_count > 0:
      print(f"Quedan undefined en columna {column}")


In [7]:
df_trabajo.dropna(subset=['children'], inplace=True)

df_trabajo.reset_index(drop=True, inplace=True)

### Refactor variable company

In [8]:
df_trabajo['is_company'] = df_trabajo['company'].notnull()

df_trabajo['is_company'] = df_trabajo['is_company'].astype(bool)

df_trabajo.drop('company', axis=1, inplace=True)

df_trabajo.rename(columns={'is_company': 'company'}, inplace=True)

In [9]:
df_trabajo['company'].value_counts()

False    58197
True      3120
Name: company, dtype: int64

### Refactor variable agent

In [10]:
df_trabajo['agent'] = df_trabajo.apply(lambda row: 0. if pd.isna(row['agent']) else row['agent'], axis=1)

In [11]:
porcentaje_nulos = (df_trabajo.isna().sum() / len(df_trabajo)) * 100

columnas_con_nulos = porcentaje_nulos[porcentaje_nulos > 0].index.tolist()

complement_percentage = 100 - porcentaje_nulos

porcentajes_df = pd.DataFrame({'Nulos': porcentaje_nulos[columnas_con_nulos], 'Complemento': complement_percentage[columnas_con_nulos]})

# No imprime nada si no quedan nulos
for columna in columnas_con_nulos:
    print(f"Columna '{columna}' tiene {porcentaje_nulos[columna]:.2f}% nulos.")

Columna 'country' tiene 0.36% nulos.


## Modificación de tipos

In [12]:
df_trabajo['hotel'] = df_trabajo['hotel'].astype('category')
df_trabajo['meal'] = df_trabajo['meal'].astype('category')
df_trabajo['country'] = df_trabajo['country'].astype('category')
df_trabajo['market_segment'] = df_trabajo['market_segment'].astype('category')
df_trabajo['distribution_channel'] = df_trabajo['distribution_channel'].astype('category')
df_trabajo['reserved_room_type'] = df_trabajo['reserved_room_type'].astype('category')
df_trabajo['assigned_room_type'] = df_trabajo['assigned_room_type'].astype('category')
df_trabajo['deposit_type'] = df_trabajo['deposit_type'].astype('category')
df_trabajo['customer_type'] = df_trabajo['customer_type'].astype('category')

In [13]:
df_trabajo['is_repeated_guest'] = df_trabajo['is_repeated_guest'].astype(bool)
df_trabajo['is_repeated_guest'].value_counts()

False    59605
True      1712
Name: is_repeated_guest, dtype: int64

In [14]:
mes_a_numero = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June' : 6,
    'July' : 7,
    'August' : 8,
    'September' : 9,
    'October' : 10,
    'November' : 11,
    'December' : 12
}

df_trabajo['arrival_date_month'] = df_trabajo['arrival_date_month'].map(mes_a_numero)

In [15]:
df_trabajo['room_changed'] = df_trabajo.apply(lambda row : row['reserved_room_type'] != row['assigned_room_type'], axis=1)
df_trabajo['room_changed'].value_counts() # Verificamos que se mantiene la cantidad de registros

False    54979
True      6338
Name: room_changed, dtype: int64

## Outliers

In [16]:
indices = df_trabajo[df_trabajo['adults'] == 0].index
df_trabajo.drop(indices, inplace=True)

In [17]:
indices = df_trabajo[df_trabajo['adults'] > 15].index
df_trabajo.drop(indices, inplace=True)

In [18]:
indices = df_trabajo[df_trabajo['children'] > 6].index
df_trabajo.drop(indices, inplace=True)

In [19]:
indices = df_trabajo[df_trabajo['babies'] > 6].index
df_trabajo.drop(indices, inplace=True)

## Ultimos cambios

Cambios realizados durante el CHP2

En un principio tomamos la desición de eliminar las filas que contenian nulos en la variable *country* ya que representaban menos del 1%, pero debido a que en el archivo de test no se puede modificar el total de filas y contenia nulos en dicha variable, decidimos modificar la variable *country* a un booleano con valor TRUE si es de portugal y FALSE en caso contrario, tomando a los nulos como que no son de portugal.

In [20]:
df_trabajo['is_prt'] = df_trabajo['country'] == 'PRT'

df_trabajo['is_prt'] = df_trabajo['is_prt'].astype(bool)

df_trabajo.drop('country', axis=1, inplace=True)

df_trabajo.rename(columns={'is_prt': 'country'}, inplace=True)

df_trabajo['country'].isna().sum()

0

In [21]:
df_trabajo['total_dias'] = df_trabajo['stays_in_week_nights'] + df_trabajo['stays_in_weekend_nights']
df_trabajo['costo_total'] = df_trabajo['adr'] * df_trabajo['total_dias']
promedio_costo_total = df_trabajo['costo_total'].mean()
df_trabajo['es_mayor_promedio_costo_total'] = df_trabajo['costo_total'] > promedio_costo_total
df_trabajo['necesita_estacionamiento'] = df_trabajo['required_car_parking_spaces'] > 0
df_trabajo['hizo_requerimientos_especiales'] = df_trabajo['total_of_special_requests'] > 0
df_trabajo['hizo_cambios'] = ((df_trabajo['booking_changes'] > 0) | (df_trabajo['total_of_special_requests'] > 0)).astype(bool)
df_trabajo['hizo_deposito'] = df_trabajo['deposit_type'] != 'No Deposit'
df_trabajo['hizo_reserva_persona'] = df_trabajo['agent'] == 0.
df_trabajo['hijos'] = df_trabajo['children'] + df_trabajo['babies']
df_trabajo['total_de_personas'] = df_trabajo['adults'] + df_trabajo['hijos']
df_trabajo.drop(['children', 'babies'], inplace=True, axis=1)
# Una especie de One Hot Encoding para la comida de la reserva
df_trabajo['tiene_desayuno'] = df_trabajo['meal'].isin(['BB', 'HB', 'FB'])
df_trabajo['tiene_almuerzo'] = df_trabajo['meal'] == 'FB'
df_trabajo['tiene_cena'] = df_trabajo['meal'].isin(['HB', 'FB'])

Agregamos estas columnas en función de los datos ya obtenidos al ver que al hacerlo, aumentar la cantidad de columnas y por la naturaleza separable de los tipos de dato de las columnas agregadas (booleanos y numèricas), el modelo entrenado logra mejores resultados al tener separaciones mucho más significativas.

In [22]:
df_trabajo.to_csv('/content/drive/MyDrive/datasetsTP/hotels_train_procesado.csv', index = False) # No guarda los indices

# Replica modificaciones en archivo de test

In [23]:
df_test = pd.read_csv('/content/drive/MyDrive/datasetsTP/hotels_test.csv')

In [24]:
df_test.isna().sum()

hotel                                 0
lead_time                             0
arrival_date_year                     0
arrival_date_month                    0
arrival_date_week_number              0
arrival_date_day_of_month             0
stays_in_weekend_nights               0
stays_in_week_nights                  0
adults                                0
children                              0
babies                                0
meal                                  0
country                              95
market_segment                        0
distribution_channel                  0
is_repeated_guest                     0
previous_cancellations                0
previous_bookings_not_canceled        0
reserved_room_type                    0
assigned_room_type                    0
booking_changes                       0
deposit_type                          0
agent                              3363
company                           25218
days_in_waiting_list                  0


In [25]:
columnas_numericas = df_test.select_dtypes(include=['int64', 'float64']).columns

for columna in columnas_numericas:
    negativos_df = df_test[df_test[columna] < 0]
    if not negativos_df.empty:
        print(f"Columna '{columna}' tiene valores negativos:")
        for index, row in negativos_df.iterrows():
            print(f"Indice: {index}, valor: {row[columna]}")
        print("\n")

## Refactor variable company

In [26]:
df_test['is_company'] = df_test['company'].notnull()

df_test['is_company'] = df_test['is_company'].astype(bool)

df_test.drop('company', axis=1, inplace=True)

df_test.rename(columns={'is_company': 'company'}, inplace=True)

In [27]:
df_test['company'].value_counts()

False    25218
True      1317
Name: company, dtype: int64

## Refactor variable agent

In [28]:
df_test['agent'] = df_test.apply(lambda row: 0. if pd.isna(row['agent']) else row['agent'], axis=1)

In [29]:
porcentaje_nulos = (df_test.isna().sum() / len(df_test)) * 100

columnas_con_nulos = porcentaje_nulos[porcentaje_nulos > 0].index.tolist()

complement_percentage = 100 - porcentaje_nulos

porcentajes_df = pd.DataFrame({'Nulos': porcentaje_nulos[columnas_con_nulos], 'Complemento': complement_percentage[columnas_con_nulos]})

for columna in columnas_con_nulos: #no imprime nada es porque no quedan nulos
    print(f"Columna '{columna}' tiene {porcentaje_nulos[columna]:.2f}% nulos.")

Columna 'country' tiene 0.36% nulos.


## Modificación de tipos

In [30]:
df_test['hotel'] = df_test['hotel'].astype('category')
df_test['meal'] = df_test['meal'].astype('category')
df_test['country'] = df_test['country'].astype('category')
df_test['market_segment'] = df_test['market_segment'].astype('category')
df_test['distribution_channel'] = df_test['distribution_channel'].astype('category')
df_test['reserved_room_type'] = df_test['reserved_room_type'].astype('category')
df_test['assigned_room_type'] = df_test['assigned_room_type'].astype('category')
df_test['deposit_type'] = df_test['deposit_type'].astype('category')
df_test['customer_type'] = df_test['customer_type'].astype('category')

In [31]:
df_test['is_repeated_guest'] = df_test['is_repeated_guest'].astype(bool)
df_test['is_repeated_guest'].value_counts()

False    25808
True       727
Name: is_repeated_guest, dtype: int64

In [32]:
mes_a_numero = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June' : 6,
    'July' : 7,
    'August' : 8,
    'September' : 9,
    'October' : 10,
    'November' : 11,
    'December' : 12
}

df_test['arrival_date_month'] = df_test['arrival_date_month'].map(mes_a_numero)

In [33]:
df_test['room_changed'] = df_test.apply(lambda row : row['reserved_room_type'] != row['assigned_room_type'], axis=1)

## Ultimos cambios

Cambios realizados durante el CHP2

In [34]:
df_test['country'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 26535 entries, 0 to 26534
Series name: country
Non-Null Count  Dtype   
--------------  -----   
26440 non-null  category
dtypes: category(1)
memory usage: 57.1 KB


In [35]:
df_test['is_prt'] = df_test['country'] == 'PRT'

df_test['is_prt'] = df_test['is_prt'].astype(bool)

df_test.drop('country', axis=1, inplace=True)

df_test.rename(columns={'is_prt': 'country'}, inplace=True)

df_test['country'].isna().sum()

0

In [36]:
df_test['total_dias'] = df_test['stays_in_week_nights'] + df_test['stays_in_weekend_nights']
df_test['costo_total'] = df_test['adr'] * df_test['total_dias']
promedio_costo_total = df_test['costo_total'].mean()
df_test['es_mayor_promedio_costo_total'] = df_test['costo_total'] > promedio_costo_total
df_test['necesita_estacionamiento'] = df_test['required_car_parking_spaces'] > 0
df_test['hizo_requerimientos_especiales'] = df_test['total_of_special_requests'] > 0
df_test['hizo_cambios'] = ((df_test['booking_changes'] > 0) | (df_test['total_of_special_requests'] > 0)).astype(bool)
df_test['hizo_deposito'] = df_test['deposit_type'] != 'No Deposit'
df_test['hizo_reserva_persona'] = df_test['agent'] == 0.
df_test['hijos'] = df_test['children'] + df_test['babies']
df_test['total_de_personas'] = df_test['adults'] + df_test['hijos']
df_test.drop(['children', 'babies'], inplace=True, axis=1)
# Una especie de One Hot Encoding para la comida de la reserva
df_test['tiene_desayuno'] = df_test['meal'].isin(['BB', 'HB', 'FB'])
df_test['tiene_almuerzo'] = df_test['meal'] == 'FB'
df_test['tiene_cena'] = df_test['meal'].isin(['HB', 'FB'])

Tiene que seguir teniendo 26535 filas

In [37]:
df_test.shape

(26535, 43)

In [38]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26535 entries, 0 to 26534
Data columns (total 43 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   hotel                           26535 non-null  category
 1   lead_time                       26535 non-null  int64   
 2   arrival_date_year               26535 non-null  int64   
 3   arrival_date_month              26535 non-null  int64   
 4   arrival_date_week_number        26535 non-null  int64   
 5   arrival_date_day_of_month       26535 non-null  int64   
 6   stays_in_weekend_nights         26535 non-null  int64   
 7   stays_in_week_nights            26535 non-null  int64   
 8   adults                          26535 non-null  int64   
 9   meal                            26535 non-null  category
 10  market_segment                  26535 non-null  category
 11  distribution_channel            26535 non-null  category
 12  is_repeated_guest 

Guardado de archivo

In [39]:
df_test.to_csv('/content/drive/MyDrive/datasetsTP/hotels_test_procesado.csv', index = False) # No guarda los indices